In [2]:
import csv
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, RDFS, OWL

# Tạo đồ thị RDF
g = Graph()

# Định nghĩa namespace
base = "http://example.com/health_qa#"
n = Namespace(base)
g.bind("", n)

# Thêm ontology declaration
g.add((URIRef(base), RDF.type, OWL.Ontology))
g.add((URIRef(base), RDFS.comment, Literal("Ontology về hỏi đáp y khoa", lang="vi")))

# Định nghĩa các lớp cơ bản
classes = {
    "Disease": "Bệnh",
    "Symptom": "Triệu chứng",
    "Treatment": "Điều trị",
    "Cause": "Nguyên nhân",
    "Question": "Câu hỏi",
    "Answer": "Câu trả lời"
}

for class_name, label in classes.items():
    class_uri = URIRef(base + class_name)
    g.add((class_uri, RDF.type, OWL.Class))
    g.add((class_uri, RDFS.label, Literal(label, lang="vi")))

# Định nghĩa các thuộc tính quan hệ
properties = {
    "hasSymptom": ("Disease", "Symptom", "Có triệu chứng"),
    "hasTreatment": ("Disease", "Treatment", "Có phương pháp điều trị"),
    "hasCause": ("Disease", "Cause", "Có nguyên nhân"),
    "hasAnswer": ("Question", "Answer", "Có câu trả lời")
}

for prop_name, (domain, range_, label) in properties.items():
    prop_uri = URIRef(base + prop_name)
    g.add((prop_uri, RDF.type, OWL.ObjectProperty))
    g.add((prop_uri, RDFS.domain, URIRef(base + domain)))
    g.add((prop_uri, RDFS.range, URIRef(base + range_)))
    g.add((prop_uri, RDFS.label, Literal(label, lang="vi")))

# Xử lý dữ liệu từ CSV
diseases = {}
questions = {}
answers = {}

with open('./medibot_project/chatbot/data/data_for_ontology.csv', 'r', encoding='utf-8') as f:  
    reader = csv.DictReader(f)
    
    for row in reader:
        disease_name = row['question']
        intent = row['intent']
        entity = row['entity']
        answer_text = row['answer']
        
        # Tạo URI cho bệnh nếu chưa có
        if disease_name not in diseases:
            disease_uri = URIRef(base + disease_name.replace(" ", ""))
            g.add((disease_uri, RDF.type, OWL.NamedIndividual))
            g.add((disease_uri, RDF.type, URIRef(base + "Disease")))
            g.add((disease_uri, RDFS.label, Literal(disease_name, lang="vi")))
            diseases[disease_name] = disease_uri
        
        # Xử lý theo loại intent
        if intent == "Triệu chứng":
            symptom_uri = URIRef(base + entity.replace(" ", ""))
            g.add((symptom_uri, RDF.type, OWL.NamedIndividual))
            g.add((symptom_uri, RDF.type, URIRef(base + "Symptom")))
            g.add((symptom_uri, RDFS.label, Literal(entity, lang="vi")))
            g.add((diseases[disease_name], URIRef(base + "hasSymptom"), symptom_uri))
            
            # Tạo câu hỏi và câu trả lời
            q_id = f"Q_{disease_name.replace(' ', '')}_Symptom"
            a_id = f"A_{disease_name.replace(' ', '')}_Symptom"
            
            q_uri = URIRef(base + q_id)
            a_uri = URIRef(base + a_id)
            
            g.add((q_uri, RDF.type, URIRef(base + "Question")))
            g.add((q_uri, RDFS.label, Literal(f"{disease_name} có triệu chứng gì?", lang="vi")))
            g.add((q_uri, URIRef(base + "hasAnswer"), a_uri))
            
            g.add((a_uri, RDF.type, URIRef(base + "Answer")))
            g.add((a_uri, RDFS.label, Literal(answer_text, lang="vi")))
            
        elif intent == "Điều trị":
            treatment_uri = URIRef(base + entity.replace(" ", ""))
            g.add((treatment_uri, RDF.type, OWL.NamedIndividual))
            g.add((treatment_uri, RDF.type, URIRef(base + "Treatment")))
            g.add((treatment_uri, RDFS.label, Literal(entity, lang="vi")))
            g.add((diseases[disease_name], URIRef(base + "hasTreatment"), treatment_uri))
            
            # Tạo câu hỏi và câu trả lời
            q_id = f"Q_{disease_name.replace(' ', '')}_Treatment"
            a_id = f"A_{disease_name.replace(' ', '')}_Treatment"
            
            q_uri = URIRef(base + q_id)
            a_uri = URIRef(base + a_id)
            
            g.add((q_uri, RDF.type, URIRef(base + "Question")))
            g.add((q_uri, RDFS.label, Literal(f"{disease_name} được điều trị như thế nào?", lang="vi")))
            g.add((q_uri, URIRef(base + "hasAnswer"), a_uri))
            
            g.add((a_uri, RDF.type, URIRef(base + "Answer")))
            g.add((a_uri, RDFS.label, Literal(answer_text, lang="vi")))
            
        elif intent == "Nguyên nhân":
            cause_uri = URIRef(base + entity.replace(" ", ""))
            g.add((cause_uri, RDF.type, OWL.NamedIndividual))
            g.add((cause_uri, RDF.type, URIRef(base + "Cause")))
            g.add((cause_uri, RDFS.label, Literal(entity, lang="vi")))
            g.add((diseases[disease_name], URIRef(base + "hasCause"), cause_uri))
            
            # Tạo câu hỏi và câu trả lời
            q_id = f"Q_{disease_name.replace(' ', '')}_Cause"
            a_id = f"A_{disease_name.replace(' ', '')}_Cause"
            
            q_uri = URIRef(base + q_id)
            a_uri = URIRef(base + a_id)
            
            g.add((q_uri, RDF.type, URIRef(base + "Question")))
            g.add((q_uri, RDFS.label, Literal(f"Nguyên nhân gây {disease_name}?", lang="vi")))
            g.add((q_uri, URIRef(base + "hasAnswer"), a_uri))
            
            g.add((a_uri, RDF.type, URIRef(base + "Answer")))
            g.add((a_uri, RDFS.label, Literal(answer_text, lang="vi")))

g.serialize(destination='./medibot_project/chatbot/data/disease_ontology.owl', format='xml', encoding='utf-8')

print("Đã chuyển đổi thành công sang OWL!")

Đã chuyển đổi thành công sang OWL!
